1. Estrutura do CRUD
* Inserir o sentimento atribuído por um usuário à notícia.
* Ler (listar) todas as notícias relacionadas a uma criptomoeda, incluindo o sentimento dos usuários.
* Atualizar o sentimento de um usuário para uma notícia.
* Excluir uma notícia e, consequentemente, os sentimentos relacionados.

In [1]:
import psycopg2
from psycopg2 import sql

def conectar():
    return psycopg2.connect(
        host="localhost",  # Endereço do servidor do banco de dados
        database="info_criptos",  # Nome do banco de dados
        user="postgres",  # Usuário do banco de dados
        password="corvinal"  # Senha do usuário
    )


## 2. Função para Inserir Notícia

In [2]:
def inserir_noticia(id_cripto, data_publicacao, tema, noticia, fonte):
    conexao = conectar()
    cursor = conexao.cursor()

    try:
        # Query para inserir notícia
        query = """
        INSERT INTO Notícias (id_cripto, data_publicacao, tema, noticia, fonte)
        VALUES (%s, %s, %s, %s, %s) RETURNING id_noticia;
        """
        cursor.execute(query, (id_cripto, data_publicacao, tema, noticia, fonte))
        id_noticia = cursor.fetchone()[0]
        conexao.commit()
        return id_noticia  # Retorna o ID da notícia recém-criada
    except Exception as e:
        conexao.rollback()
        raise e
    finally:
        cursor.close()
        conexao.close()


## Função para Inserir Sentimento

In [3]:
def inserir_sentimento(id_noticia, id_usuario, sentimento, score_sentimento):
    conexao = conectar()
    cursor = conexao.cursor()

    try:
        # Query para inserir sentimento
        query = """
        INSERT INTO Sentimentos_Notícias (id_noticia, id_usuario, sentimento, score_sentimento)
        VALUES (%s, %s, %s, %s);
        """
        cursor.execute(query, (id_noticia, id_usuario, sentimento, score_sentimento))
        conexao.commit()
        return f"Sentimento inserido para a notícia {id_noticia}."
    except Exception as e:
        conexao.rollback()
        raise e
    finally:
        cursor.close()
        conexao.close()


## Função para Listar Notícias e Sentimentos

In [4]:
def listar_noticias_por_criptomoeda(id_cripto):
    conexao = conectar()
    cursor = conexao.cursor()

    try:
        # Query para listar notícias e sentimentos
        query = """
        SELECT N.id_noticia, N.tema, N.noticia, U.nome AS usuario, S.sentimento, S.score_sentimento
        FROM Notícias N
        JOIN Sentimentos_Notícias S ON N.id_noticia = S.id_noticia
        JOIN Usuarios U ON S.id_usuario = U.id_usuario
        WHERE N.id_cripto = %s;
        """
        cursor.execute(query, (id_cripto,))
        resultados = cursor.fetchall()

        # Formatar resultados
        noticias = []
        for linha in resultados:
            noticias.append({
                "id_noticia": linha[0],
                "tema": linha[1],
                "noticia": linha[2],
                "usuario": linha[3],
                "sentimento": linha[4],
                "score_sentimento": linha[5]
            })
        return noticias
    except Exception as e:
        raise e
    finally:
        cursor.close()
        conexao.close()


## Função para Atualizar Sentimento

In [5]:
def atualizar_sentimento(id_noticia, id_usuario, novo_sentimento, novo_score):
    conexao = conectar()
    cursor = conexao.cursor()

    try:
        # Query para atualizar sentimento
        query = """
        UPDATE Sentimentos_Notícias
        SET sentimento = %s, score_sentimento = %s
        WHERE id_noticia = %s AND id_usuario = %s;
        """
        cursor.execute(query, (novo_sentimento, novo_score, id_noticia, id_usuario))
        conexao.commit()
        return f"Sentimento atualizado para a notícia {id_noticia} pelo usuário {id_usuario}."
    except Exception as e:
        conexao.rollback()
        raise e
    finally:
        cursor.close()
        conexao.close()


## Função para Excluir Notícia

In [6]:
def excluir_noticia(id_noticia):
    conexao = conectar()
    cursor = conexao.cursor()

    try:
        # Excluir sentimentos relacionados
        query_sentimentos = """
        DELETE FROM Sentimentos_Notícias WHERE id_noticia = %s;
        """
        cursor.execute(query_sentimentos, (id_noticia,))

        # Excluir notícia
        query_noticia = """
        DELETE FROM Notícias WHERE id_noticia = %s;
        """
        cursor.execute(query_noticia, (id_noticia,))
        conexao.commit()
        return f"Notícia {id_noticia} e seus sentimentos relacionados foram excluídos."
    except Exception as e:
        conexao.rollback()
        raise e
    finally:
        cursor.close()
        conexao.close()


## Inserir uma notícia e seu sentimento

In [7]:
id_noticia = inserir_noticia(
    id_cripto=1,
    data_publicacao="2025-01-01",
    tema="Regulação",
    noticia="Bitcoin será regulamentado em breve.",
    fonte="CoinDesk"
)

resultado_sentimento = inserir_sentimento(
    id_noticia=id_noticia,
    id_usuario=1,
    sentimento="Positivo",
    score_sentimento=0.8
)

print(resultado_sentimento)


Sentimento inserido para a notícia 6.


## Listar notícias relacionadas a uma criptomoeda

In [12]:
noticias = listar_noticias_por_criptomoeda(id_cripto=1)
for noticia in noticias:
    print(noticia)

{'id_noticia': 6, 'tema': 'Regulação', 'noticia': 'Bitcoin será regulamentado em breve.', 'usuario': 'João Silva', 'sentimento': 'Positivo', 'score_sentimento': Decimal('0.80')}


##  Atualizar o sentimento de uma notícia

In [9]:
resultado = atualizar_sentimento(
    id_noticia=1,
    id_usuario=1,
    novo_sentimento="Negativo",
    novo_score=-0.5
)
print(resultado)


Sentimento atualizado para a notícia 1 pelo usuário 1.


In [ ]:
noticias = listar_noticias_por_criptomoeda(id_cripto=1)
for noticia in noticias:
    print(noticia)

## Excluir uma notícia

In [11]:
resultado = excluir_noticia(id_noticia=1)
print(resultado)

Notícia 1 e seus sentimentos relacionados foram excluídos.


In [ ]:
noticias = listar_noticias_por_criptomoeda(id_cripto=1)
for noticia in noticias:
    print(noticia)

## Create

In [ ]:
"""-- Tabela Criptomoedas
CREATE TABLE Criptomoedas (
    id_cripto SERIAL PRIMARY KEY, -- Identificador único da criptomoeda
    nome VARCHAR(50) NOT NULL, -- Nome da criptomoeda
    simbolo VARCHAR(10) NOT NULL, -- Símbolo da criptomoeda (ex.: BTC, ETH)
    descricao TEXT, -- Descrição da criptomoeda
    mercado VARCHAR(50) -- Nome do mercado onde a criptomoeda é negociada
);

COMMENT ON TABLE Criptomoedas IS 'Tabela que armazena informações sobre criptomoedas, incluindo nome, símbolo e mercado de negociação.';
COMMENT ON COLUMN Criptomoedas.id_cripto IS 'Identificador único da criptomoeda';
COMMENT ON COLUMN Criptomoedas.nome IS 'Nome da criptomoeda';
COMMENT ON COLUMN Criptomoedas.simbolo IS 'Símbolo da criptomoeda (ex.: BTC, ETH)';
COMMENT ON COLUMN Criptomoedas.descricao IS 'Descrição da criptomoeda';
COMMENT ON COLUMN Criptomoedas.mercado IS 'Nome do mercado onde a criptomoeda é negociada';

-- Tabela Cotações
CREATE TABLE Cotações (
    id_cotacao SERIAL PRIMARY KEY, -- Identificador único da cotação
    id_cripto INT REFERENCES Criptomoedas (id_cripto), -- Referência à criptomoeda
    data_hora TIMESTAMP NOT NULL, -- Data e hora da cotação
    preco DECIMAL(18, 8) NOT NULL, -- Preço da criptomoeda
    volume DECIMAL(18, 2), -- Volume negociado
    market_cap DECIMAL(18, 2), -- Capitalização de mercado
    variacao DECIMAL(5, 2) -- Variação percentual do preço
);

COMMENT ON TABLE Cotações IS 'Tabela que armazena informações sobre cotações das criptomoedas, incluindo preço, volume e market cap.';
COMMENT ON COLUMN Cotações.id_cotacao IS 'Identificador único da cotação';
COMMENT ON COLUMN Cotações.id_cripto IS 'Referência à criptomoeda';
COMMENT ON COLUMN Cotações.data_hora IS 'Data e hora da cotação';
COMMENT ON COLUMN Cotações.preco IS 'Preço da criptomoeda';
COMMENT ON COLUMN Cotações.volume IS 'Volume negociado';
COMMENT ON COLUMN Cotações.market_cap IS 'Capitalização de mercado';
COMMENT ON COLUMN Cotações.variacao IS 'Variação percentual do preço';

-- Tabela Notícias
CREATE TABLE Notícias (
    id_noticia SERIAL PRIMARY KEY, -- Identificador único da notícia
    id_cripto INT REFERENCES Criptomoedas (id_cripto), -- Referência à criptomoeda
    data_publicacao DATE NOT NULL, -- Data de publicação da notícia
    tema VARCHAR(50), -- Tema principal da notícia
    noticia TEXT NOT NULL, -- Texto completo da notícia
    fonte VARCHAR(100) -- Fonte da notícia (ex.: Bloomberg, CoinDesk)
);

COMMENT ON TABLE Notícias IS 'Tabela que armazena notícias relacionadas às criptomoedas.';
COMMENT ON COLUMN Notícias.id_noticia IS 'Identificador único da notícia';
COMMENT ON COLUMN Notícias.id_cripto IS 'Referência à criptomoeda';
COMMENT ON COLUMN Notícias.data_publicacao IS 'Data de publicação da notícia';
COMMENT ON COLUMN Notícias.tema IS 'Tema principal da notícia';
COMMENT ON COLUMN Notícias.noticia IS 'Texto completo da notícia';
COMMENT ON COLUMN Notícias.fonte IS 'Fonte da notícia (ex.: Bloomberg, CoinDesk)';

-- Tabela Usuários
CREATE TABLE Usuarios (
    id_usuario SERIAL PRIMARY KEY, -- Identificador único do usuário
    nome VARCHAR(100) NOT NULL, -- Nome do usuário
    email VARCHAR(100) UNIQUE NOT NULL, -- Email do usuário (deve ser único)
    senha VARCHAR(255) NOT NULL, -- Senha do usuário (criptografada)
    admin_flag BOOLEAN DEFAULT FALSE -- Indica se o usuário é administrador
);

COMMENT ON TABLE Usuarios IS 'Tabela que registra os usuários do sistema, incluindo permissões administrativas.';
COMMENT ON COLUMN Usuarios.id_usuario IS 'Identificador único do usuário';
COMMENT ON COLUMN Usuarios.nome IS 'Nome do usuário';
COMMENT ON COLUMN Usuarios.email IS 'Email do usuário (deve ser único)';
COMMENT ON COLUMN Usuarios.senha IS 'Senha do usuário (criptografada)';
COMMENT ON COLUMN Usuarios.admin_flag IS 'Indica se o usuário é administrador';


-- Tabela Sentimentos de Notícias
CREATE TABLE Sentimentos_Notícias (
    id_sentimento SERIAL PRIMARY KEY, -- Identificador único do sentimento
    id_noticia INT REFERENCES Notícias (id_noticia), -- Referência à notícia
    id_usuario INT REFERENCES Usuarios (id_usuario), -- Referência ao usuário
    sentimento VARCHAR(20), -- Classificação do sentimento (positivo, neutro, negativo)
    score_sentimento DECIMAL(5, 2) -- Escore numérico do sentimento (-1 a 1)
);

COMMENT ON TABLE Sentimentos_Notícias IS 'Tabela que armazena os sentimentos associados às notícias sobre criptomoedas, incluindo quem atribuiu o sentimento.';
COMMENT ON COLUMN Sentimentos_Notícias.id_sentimento IS 'Identificador único do sentimento';
COMMENT ON COLUMN Sentimentos_Notícias.id_noticia IS 'Referência à notícia';
COMMENT ON COLUMN Sentimentos_Notícias.id_usuario IS 'Referência ao usuário que atribuiu o sentimento';
COMMENT ON COLUMN Sentimentos_Notícias.sentimento IS 'Classificação do sentimento (positivo, neutro, negativo)';
COMMENT ON COLUMN Sentimentos_Notícias.score_sentimento IS 'Escore numérico do sentimento (-1 a 1)';

-- Tabela Transações de Mercado
CREATE TABLE Transações_Mercado (
    id_transacao SERIAL PRIMARY KEY, -- Identificador único da transação
    id_cripto INT REFERENCES Criptomoedas (id_cripto), -- Referência à criptomoeda
    data_hora TIMESTAMP NOT NULL, -- Data e hora da transação
    tipo VARCHAR(20) NOT NULL, -- Tipo da transação (compra, venda)
    quantidade DECIMAL(18, 8) NOT NULL, -- Quantidade negociada
    preco_unitario DECIMAL(18, 8) NOT NULL -- Preço unitário da criptomoeda
);

COMMENT ON TABLE Transações_Mercado IS 'Tabela que registra as transações realizadas no mercado de criptomoedas.';
COMMENT ON COLUMN Transações_Mercado.id_transacao IS 'Identificador único da transação';
COMMENT ON COLUMN Transações_Mercado.id_cripto IS 'Referência à criptomoeda';
COMMENT ON COLUMN Transações_Mercado.data_hora IS 'Data e hora da transação';
COMMENT ON COLUMN Transações_Mercado.tipo IS 'Tipo da transação (compra, venda)';
COMMENT ON COLUMN Transações_Mercado.quantidade IS 'Quantidade negociada';
COMMENT ON COLUMN Transações_Mercado.preco_unitario IS 'Preço unitário da criptomoeda';

-- Tabela Ordens de Compra/Venda
CREATE TABLE Ordens (
    id_ordem SERIAL PRIMARY KEY, -- Identificador único da ordem
    id_cripto INT REFERENCES Criptomoedas (id_cripto), -- Referência à criptomoeda
    tipo VARCHAR(20) NOT NULL, -- Tipo da ordem (compra, venda)
    quantidade DECIMAL(18, 8) NOT NULL, -- Quantidade desejada
    preco_limite DECIMAL(18, 8) NOT NULL -- Preço limite da ordem
);

COMMENT ON TABLE Ordens IS 'Tabela que registra as ordens de compra ou venda de criptomoedas.';
COMMENT ON COLUMN Ordens.id_ordem IS 'Identificador único da ordem';
COMMENT ON COLUMN Ordens.id_cripto IS 'Referência à criptomoeda';
COMMENT ON COLUMN Ordens.tipo IS 'Tipo da ordem (compra, venda)';
COMMENT ON COLUMN Ordens.quantidade IS 'Quantidade desejada';
COMMENT ON COLUMN Ordens.preco_limite IS 'Preço limite da ordem';

-- Tabela Tendências de Preço
CREATE TABLE Tendências_Preço (
    id_tendencia SERIAL PRIMARY KEY, -- Identificador único da tendência
    id_cripto INT REFERENCES Criptomoedas (id_cripto), -- Referência à criptomoeda
    periodo VARCHAR(20), -- Período analisado (ex.: 24h, 7d)
    variacao_preco DECIMAL(5, 2), -- Variação percentual do preço
    tendencia VARCHAR(20) -- Classificação da tendência (alta, baixa, estável)
);

COMMENT ON TABLE Tendências_Preço IS 'Tabela que armazena informações sobre tendências de preços de criptomoedas.';
COMMENT ON COLUMN Tendências_Preço.id_tendencia IS 'Identificador único da tendência';
COMMENT ON COLUMN Tendências_Preço.id_cripto IS 'Referência à criptomoeda';
COMMENT ON COLUMN Tendências_Preço.periodo IS 'Período analisado (ex.: 24h, 7d)';
COMMENT ON COLUMN Tendências_Preço.variacao_preco IS 'Variação percentual do preço';
COMMENT ON COLUMN Tendências_Preço.tendencia IS 'Classificação da tendência (alta, baixa, estável)';

-- Tabela Dados Externos
CREATE TABLE Dados_Externos (
    id_dado SERIAL PRIMARY KEY, -- Identificador único do dado externo
    descricao TEXT, -- Descrição do fator externo (ex.: taxa de juros)
    data_hora TIMESTAMP NOT NULL, -- Data e hora da coleta
    valor DECIMAL(18, 8) -- Valor do fator externo
);

COMMENT ON TABLE Dados_Externos IS 'Tabela que armazena dados externos que podem impactar o mercado de criptomoedas.';
COMMENT ON COLUMN Dados_Externos.id_dado IS 'Identificador único do dado externo';
COMMENT ON COLUMN Dados_Externos.descricao IS 'Descrição do fator externo (ex.: taxa de juros)';
COMMENT ON COLUMN Dados_Externos.data_hora IS 'Data e hora da coleta';
COMMENT ON COLUMN Dados_Externos.valor IS 'Valor do fator externo';

-- Tabela Imagens de Criptomoedas
CREATE TABLE Imagens_Criptomoedas (
    id_imagem SERIAL PRIMARY KEY, -- Identificador único da imagem
    id_cripto INT REFERENCES Criptomoedas (id_cripto), -- Referência à criptomoeda correspondente
    tipo VARCHAR(50) NOT NULL, -- Tipo da imagem (ex.: logo, ícone)
    conteudo BYTEA NOT NULL, -- Conteúdo da imagem armazenado como binário
    data_upload TIMESTAMP NOT NULL -- Data de upload da imagem
);

COMMENT ON TABLE Imagens_Criptomoedas IS 'Tabela que armazena imagens relacionadas às criptomoedas, como logos ou ícones.';
COMMENT ON COLUMN Imagens_Criptomoedas.id_imagem IS 'Identificador único da imagem';
COMMENT ON COLUMN Imagens_Criptomoedas.id_cripto IS 'Referência à criptomoeda correspondente';
COMMENT ON COLUMN Imagens_Criptomoedas.tipo IS 'Tipo da imagem (ex.: logo, ícone)';
COMMENT ON COLUMN Imagens_Criptomoedas.conteudo IS 'Conteúdo da imagem armazenado como binário';
COMMENT ON COLUMN Imagens_Criptomoedas.data_upload IS 'Data de upload da imagem';
"""

## Inserir Dados na Tabela Criptomoedas

In [ ]:
"""INSERT INTO Criptomoedas (nome, simbolo, descricao, mercado)
VALUES 
('Bitcoin', 'BTC', 'Primeira criptomoeda descentralizada', 'Binance'),
('Ethereum', 'ETH', 'Plataforma de contratos inteligentes', 'Coinbase'),
('Cardano', 'ADA', 'Criptomoeda de terceira geração focada em escalabilidade', 'Kraken'),
('Solana', 'SOL', 'Plataforma blockchain para aplicativos descentralizados', 'Binance'),
('Ripple', 'XRP', 'Criptomoeda para pagamentos digitais', 'Bitstamp');
"""

## Inserir Dados na Tabela Cotações

In [ ]:
"""INSERT INTO Cotações (id_cripto, data_hora, preco, volume, market_cap, variacao)
VALUES 
(1, '2025-01-01 12:00:00', 27000.50, 1000000.00, 500000000.00, 2.5),
(2, '2025-01-01 12:00:00', 1900.75, 500000.00, 250000000.00, -1.8),
(3, '2025-01-01 12:00:00', 0.40, 200000.00, 12000000.00, 5.2),
(4, '2025-01-01 12:00:00', 23.70, 300000.00, 70000000.00, -0.5),
(5, '2025-01-01 12:00:00', 0.50, 150000.00, 20000000.00, 1.2);
"""

## Inserir Dados na Tabela Transações_Mercado

In [ ]:
"""INSERT INTO Transações_Mercado (id_cripto, data_hora, tipo, quantidade, preco_unitario)
VALUES 
(1, '2025-01-01 12:30:00', 'Compra', 0.1, 27000.50),
(2, '2025-01-01 13:00:00', 'Venda', 0.2, 1900.75),
(3, '2025-01-01 13:30:00', 'Compra', 100, 0.40),
(4, '2025-01-01 14:00:00', 'Venda', 50, 23.70),
(5, '2025-01-01 14:30:00', 'Compra', 200, 0.50);
"""

## Inserir Dados na Tabela Ordens

In [ ]:
"""INSERT INTO Ordens (id_cripto, tipo, quantidade, preco_limite)
VALUES 
(1, 'Compra', 0.05, 25000.00),
(2, 'Venda', 0.10, 2000.00),
(3, 'Compra', 200, 0.35),
(4, 'Venda', 50, 25.00),
(5, 'Compra', 100, 0.45);
"""

## Inserir Dados na Tabela Tendências_Preço

In [ ]:
"""INSERT INTO Tendências_Preço (id_cripto, periodo, variacao_preco, tendencia)
VALUES 
(1, '24h', 2.5, 'Alta'),
(2, '24h', -1.8, 'Baixa'),
(3, '24h', 5.2, 'Alta'),
(4, '24h', -0.5, 'Estável'),
(5, '24h', 1.2, 'Alta');
"""

## Inserir Dados na Tabela Dados_Externos

In [ ]:
"""INSERT INTO Dados_Externos (descricao, data_hora, valor)
VALUES 
('Taxa de juros dos EUA', '2025-01-01 09:00:00', 0.05),
('Índice de inflação global', '2025-01-01 09:00:00', 3.2),
('Preço do ouro', '2025-01-01 09:00:00', 1850.00),
('Preço do barril de petróleo', '2025-01-01 09:00:00', 75.50),
('Taxa de câmbio USD/EUR', '2025-01-01 09:00:00', 1.10);
"""

##  Inserir Dados na Tabela Notícias

In [ ]:
"""INSERT INTO Notícias (id_cripto, data_publicacao, tema, noticia, fonte)
VALUES 
(1, '2025-01-01', 'Regulação', 'Bitcoin será regulamentado em breve.', 'CoinDesk'),
(2, '2025-01-01', 'Adoção', 'Ethereum será aceito como pagamento em grandes varejistas.', 'Bloomberg'),
(3, '2025-01-01', 'Tecnologia', 'Cardano anuncia atualização para maior escalabilidade.', 'CryptoNews'),
(4, '2025-01-01', 'Desempenho', 'Solana apresenta alta no volume de transações.', 'Kraken Blog'),
(5, '2025-01-01', 'Parceria', 'Ripple firma parceria com bancos europeus.', 'Bitstamp Insights');
"""

## Inserir Dados na Tabela Usuários

In [ ]:
"""INSERT INTO Usuarios (nome, email, senha, admin_flag)
VALUES 
('João Silva', 'joao@gmail.com', 'senha123', TRUE),
('Maria Oliveira', 'maria@gmail.com', 'senha123', FALSE),
('Carlos Santos', 'carlos@gmail.com', 'senha123', FALSE),
('Ana Costa', 'ana@gmail.com', 'senha123', FALSE),
('Pedro Almeida', 'pedro@gmail.com', 'senha123', TRUE);
"""

## Inserir Dados na Tabela Sentimentos_Notícias

In [ ]:
"""INSERT INTO Sentimentos_Notícias (id_noticia, id_usuario, sentimento, score_sentimento)
VALUES 
(1, 1, 'Positivo', 0.8),
(2, 2, 'Neutro', 0.0),
(3, 3, 'Positivo', 0.6),
(4, 4, 'Negativo', -0.4),
(5, 5, 'Positivo', 0.9);
"""

## Inserir Dados na Tabela Imagens_Criptomoedas

### Tem que inserir dados nessa tabela ainda, não sei como que faz.